In [2]:
# -*- coding: utf-8 -*-
#在1m文件中标识range
import pandas as pd
df1m=pd.read_csv("D:\\002 MakeLive\DataCollection\\bar data\SHFE.RB\\SHFE.RB 60.csv",index_col='strtime')
df10m=pd.read_csv("D:\\002 MakeLive\DataCollection\\bar data\SHFE.RB\\SHFE.RB 600.csv")
time10m=df10m.strtime
df1m['range']=0
for t in time10m:
    #print t
    if "14:55" in t:
        r=5
    else:r=10
    df1m.ix[t,"range"]=r
df1m.to_csv("SHFE.RB 60_range.csv")

C:\Python27\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


In [3]:
# -*- coding: utf-8 -*-
'''
TR := MAX(MAX(HIGH-LOW,ABS(HIGH-REF(CLOSE,1))),ABS(LOW-REF(CLOSE,1)))
#计算每根10mK的TR
'''
import pandas as pd
import numpy as np
df10m=pd.read_csv("D:\\002 MakeLive\DataCollection\\bar data\SHFE.RB\\SHFE.RB 600.csv")

high=df10m.high
low=df10m.low
close=df10m.close
closeshift1=close.shift(1).fillna(0)
c=high-low
d=np.abs(high-closeshift1)
df1=pd.DataFrame({'c':c,'d':d})
df1['A']=df1.max(axis=1)
df1.drop('c',axis=1,inplace=True)
df1.drop('d',axis=1,inplace=True)
df1['B']=np.abs(low-closeshift1)
df10m['TR']=df1.max(axis=1)
df10m.to_csv("SHFE.RB 600_TR.csv")
print "finished!"


finished!


In [1]:
# -*- coding: utf-8 -*-
'''
计算1km的TR
2.0
i从beginindex=1开始遍历1mK，取range值,以及i-1 K线的的close作为preclose，取i的open作为open
遍历range值（r in range)，取i~r的high.max和low.min作为最高最低值
结合preclose和open，计算TR
===============================================
1.0
遍历10mK，根据10mK取对应1mK的range值
根据range值取1mK
遍历1mK，生成截至该1mK时该10mK的值
取上一根10M的close一起计算每根1M根的TR值
取前N根10mK与当前TR，一起计算当前1mk的ATR
'''
import pandas as pd
import numpy as np
df1m=pd.read_csv("SHFE.RB 60_range.csv")
df1m['TR']=0
beginindex=10
num=df1m.shape[0]
i=beginindex
while(i<num):
    preclose=df1m.ix[i-1,'close']
    rng=df1m.ix[i,'range']
    if rng==0:
        i+=1
        continue
    hm=df1m.ix[i,'high']
    lm=df1m.ix[i,'low']
    df1m.ix[i,'TR']=max((hm-lm),abs(hm-preclose),abs(lm-preclose))
    for r in range(1,rng):
        hm=max(hm,df1m.ix[i+r,'high'])
        lm=min(lm,df1m.ix[i+r,'low'])
        df1m.ix[i+r,'TR']=max((hm-lm),abs(hm-preclose),abs(lm-preclose))
    i+=rng
df1m.to_csv("SHFE.RB 60_range_TR.csv")
print "finished"

C:\Python27\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [ ]:
# -*- coding: utf-8 -*-
'''
计算1mk的ATR
从N-1开始遍历10mk，取0~N-2的ATR:trlist
根据N的strtime定位到1mk，取rng,1mkdata
for i in range(rng):
    trlist.append(1mkdata.iloc[i]['TR']
    df1m['ATR']=trlist.mean()
    del trlist[-1]
'''
import pandas as pd
N=20
df10m=pd.read_csv("SHFE.RB 600_TR.csv")
atlist=df10m[:N,'TR'].tolist()
print atlist